# The SeaFlux dataset 

The SeaFlux data set allows for the homogenization of air-sea CO$_2$ flux calculations. 
To minimize the potential differences that may arise during calculation, we've made a product available at 1°⨉1° by monthly resolution for 1988-2018. 

Here, you'll find the way you can download the data and quickly calculate air-sea CO$_2$ fluxes when you only have $p$CO$_2$ based on the bulk flux formulation:
$$
F\text{CO}_2 = K_0 \cdot k_w \cdot (p\text{CO}_2^{sea} - p\text{CO}_2^{sea}) \cdot ice^{free}
$$

Where 
$K_0$ is the solubility of CO$_2$ in seawater, 
$k_w$ is the gas transfer velocity using the formulation of Wanninkhof (1992) that has a square dependence on wind speed (second moment of the wind),
$p\text{CO}_2^{sea}$ is the oceanic partial pressure of carbon dioxide in the surface ocean, 
$p\text{CO}_2^{atm}$ is the atmospheric partial pressure of carbon dioxide in the marine boundary layer, 
$ice^{free}$ is the fraction of open ocean in a particular grid cell. 

Here we tackle the differences that arise in each of these components. In this notebook, we'll detail how to implement these corrections using Python. 

In [1]:
# cell will be hidden
%load_ext autoreload
%autoreload 2

import warnings
from matplotlib import pyplot as plt
import xarray as xr
import sys

sys.path.insert(0, '..')

plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = [8, 3.5]
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
%pylab inline

# for the rest of the document seaflux is referred to as sf
import seaflux as sf 
import xarray as xr

Populating the interactive namespace from numpy and matplotlib


## pCO2 area coverage

This correction applies to data-based surface $p\text{CO}_2^{sea}$ products that provide pseudo-global coverage.
However, due to various implementations and predictor variables used by each of these data-based products, there is a difference in the coverage of these products. 
This is true primarily for the coastal ocean and seasonally ice covered regions. 

Recent work by [Landschützer et al. (2020)](https://doi.org/10.5194/essd-12-2537-2020) provides a monthly $p\text{CO}_2^{sea}$ climatology that is available 
for open ocean, coastal ocean, and seasonally ice-covered seas. 
By scaling this product, SeaFlux is able to provide a data product that can fill $p\text{CO}_2^{sea}$ for the pseudo-global data-based products. 

For details on this process, please see [Fay et al. (2021)](https://doi.org/10.5194/essd-2021-16).

### Open example data set
I use the [CSIR-ML6 product](https://gmd.copernicus.org/articles/12/5113/2019/) to demonstrate the data filling procedure. 
I have pre-downloaded the data into the folder `../data/example`. 

In [3]:
# this cell will be hidden
from seaflux.data.download_zenodo_files import fetch_data as download_netcdfs

# fetch the CSIR-ML6 data as an example. The data is stored on Figshare 
csir_data = dict(spco2='https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/24441425/CSIRML6_CO2_19822019_figshare_v2020.6.nc')

# Here i repurpose the 
csir_spco2 = download_netcdfs('../data/example/', **csir_data, keep_attrs=True)['spco2']

In [3]:
csir_fname = '../data/example/CSIRML6_CO2_19822019_figshare_v2020.6.nc'
csir = xr.open_mfdataset(csir_fname, preprocess=sf.data.utils.preprocess())
csir_spco2 = csir.spco2

### Download scaled climatology
The scaled climatology is downloadable directly at Zenodo (https://doi.org/10.5281/zenodo.4133802).  
However, by using the SeaFlux tool to download the data you can be sure that you'll always have the 
latest version of the data.  
The function downloads the data and returns it as an `xr.Dataset` which provides a useful interface
to view the data. 

In [4]:
# download climatological filling product. Have a look at the urls 
# by looking at the documentation for the function
scaled = sf.data.scaled_spco2_for_filling(dest_path='../data/output/')

In [5]:
scaled

<xarray.Dataset>
Dimensions:            (lat: 180, lon: 360, time: 372)
Coordinates:
  * lat                (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time               (time) datetime64[ns] 1988-01-15 ... 2018-12-15
Data variables:
    spco2_clim_scaled  (time, lat, lon) float64 dask.array<chunksize=(12, 90, 180), meta=np.ndarray>
    scaling_factor     (time) float64 dask.array<chunksize=(12,), meta=np.ndarray>
Attributes:
    source:    https://zenodo.org/record/4664653
    doi:       https://doi.org/10.5281/zenodo.4133802
    contact:   luke.gregor@usys.ethz.ch
    citation:  Fay, A. R., Gregor, L., Landschützer, P., McKinley, G. A., Gru...
    history:   Downloaded with SeaFlux code for Python on 2021-04-06

### Filling the example data
The missing regions from the previous plot are not missing in the scaled product. 

In [6]:
csir_filled = csir_spco2.fillna(scaled.spco2_clim_scaled)

In [7]:
# this cell will not show

props = dict(cmap=plt.cm.Spectral_r, levels=np.arange(280, 461, 10), add_colorbar=False)

fig = plt.figure(figsize=[6, 8])
imgs = [
    csir_spco2.mean('time').plot.contourf(
        ax=sf.data.utils.subplot_map(311, land_color='k'), **props),
    scaled.spco2_clim_scaled.mean('time').plot.contourf(
        ax=sf.data.utils.subplot_map(312, land_color='k'), **props),
    csir_filled.mean('time').plot.contourf(
        ax=sf.data.utils.subplot_map(313, land_color='k'), **props)
]

imgs[0].axes.set_title('Pseudo-global coverage of CSIR-ML6')
imgs[1].axes.set_title('Global coverage of SeaFlux scaled MPI-ULB-SOMFFN')
imgs[2].axes.set_title('CSIR-ML6 filled with SeaFlux scaled climatology')

fig.tight_layout()
cb = plt.colorbar(imgs[0], ax=[img.axes for img in imgs], fraction=0.03).set_label('pCO₂ (µatm)')

fig.savefig('./img/seaflux_csir_pco2_filling.png', bbox_inches='tight', dpi=90)
plt.close()

The figure below shows average pCO2 for the unfilled (CSIR), filler (MPI-ULB-SOMFFN scaled), and filled products (unfilled + filler).
<div style="display: block; margin-left: auto; margin-right: auto; width: 70%;">
    <img src="./img/seaflux_csir_pco2_filling.png">
</div>

## Fluxes

Similarly to the pCO$_2$ filling, the data first has to be downloaded. 
This might take some time, particularly since the file for $k_w$ is calculated for five wind products, but this only has to be done once.
Further, there is a progress bar that will show for each of the files that is downloaded. 
You can specify where the data are downloaded to, making it easy to find the data later. 
The default is the `Downloads` folder in your home directory.

In [7]:
# download flux data
flux_data = sf.data.flux_calc_data(dest_path='../data/output/')

## Unit analysis
The SeaFlux product provides the remaining parameters to calculate fluxes. 
This means that the calculation can be a simple multiplication. 
However, units have to be taken care of. 
Below we show a table of the units for each of the SeaFlux variables used in the bulk flux calculation. 

| Variable              | SeaFlux units     | Transformation          | Output units      |
|:----------------------|:------------------|:------------------------|:------------------|
| $\Delta p\text{CO}_2$ | µatm              |                         | µatm              |
| $K_0$                 | mol m$^{-3}$ µatm$^{-1}$ |                  | mol m$^{-3}$ µatm$^{-1}$ |
| $k_w$                 | cm hr$^{-1}$      | $\times \frac{24}{100}$ | m day$^{-1}$      |
| ice$^{conc}$          | ice fraction      | $1 - ice^{conc}$        | ice free fraction |
| area                  | m$^2$             |                         | m$^2$             |
|                       |                   |                         |                   |
| **BULK FLUXES**       |                   |                         |                   |
| $F\text{CO}_2$        |                   | $K_0 \cdot k_w \cdot \Delta pCO_2 \cdot$ ice| molC m$^{-2}$ d$^{-1}$ |
| **INTEGRATION**       |                   |                         |                   |
| $F\text{CO}_2^{int}$  | molC m$^{-2}$ d$^{-1}$ | $\times$ (m$^2 \cdot$ 12.01 g mol$^{-1}$ $\cdot$ 365 d yr$^{-1}$)| gC yr$^{-1}$ |

In [8]:
ds = flux_data.sel(wind='ERA5').drop('wind').load()

# assigning variables and performing unit transformations
kw = ds.kw_scaled * (24/100)      # cm/hr --> m/d
K0 = ds.sol_Weiss74               # mol/m3/uatm
dpco2 = csir_filled - ds.pCO2atm  # uatm
ice_free = 1 - ds.ice.fillna(0)
area = ds.area                    # m2

# bulk flux calculation
flux_mol_m2_day = kw * K0 * dpco2 * ice_free
flux_avg_yr = flux_mol_m2_day.mean('time') * 365  # molC/m2/year
flux_integrated = (flux_mol_m2_day * area * 365 * 12.011).sum(dim=['lat', 'lon'])  # gC/year

In [9]:
# this cell will not show

fig = plt.figure(figsize=[6, 2.8], dpi=120)
img = flux_avg_yr.plot.contourf(
    ax=sf.data.utils.subplot_map(land_color='k'), 
    levels=21, 
    add_colorbar=False,
    robust=True)

fig.tight_layout()
cb = plt.colorbar(img, ax=[img.axes], location='left', pad=0.02, fraction=0.07)

img.colorbar.set_label('$FCO_2$ (mol m$^{-2}$ yr$^{-1}$)')
img.axes.set_title('')
# img.axes.set_title('Average air sea CO$_2$ fluxes using ERA5 wind')

fig.savefig('./img/seaflux_csir_era5_avg_flux_map.png', bbox_inches='tight', dpi=120)
plt.close()

In [16]:
import pandas as pd

In [24]:
# this cell will not show

flux_integrated.load()

fig, ax = plt.subplots(figsize=[6, 2.6], dpi=120)

(flux_integrated * 1e-15).resample(time='1AS', loffset=pd.Timedelta(365//2, 'D')).mean().plot(lw=5, zorder=3, ax=ax)
(flux_integrated * 1e-15).plot(ax=ax, lw=1, zorder=3, alpha=0.5, color='C0')

sf.data.utils.style_line_subplot(ax)

ax.set_ylabel('$F$CO$_2$ (Pg C yr$^{-1}$)')
# ax.set_title('Globally integrated CO$_2$ fluxes for filled CSIR-ML6')

fig.tight_layout()
fig.savefig('./img/seaflux_csir_era5_integrated_fluxes.png')
plt.close()

The figure below shows the output for the fluxes calculated using the SeaFlux data. 
Notice that the ice covered regions, particularly the Arctic, have low air-sea CO$_2$ fluxes.
The output has been multiplied by 365 days/year to convert the flux to $mol C\ m^{-1}\, yr^{-2}$.

The bottom figure shows the globally integrated air-sea CO$_2$ fluxes. 
Here, the fluxes have been multiplied by the area ($m^2$) and converted to $gC\ yr^{-1}$ (using the conversion shown in the table above). 

<div style="display: block; margin-left: auto; margin-right: auto; width: 70%; word-wrap: break-word">
    <img src="./img/seaflux_csir_era5_avg_flux_map.png">
    <img src="./img/seaflux_csir_era5_integrated_fluxes.png">
</div>